
## Instruct Fine-tuning [Gemma](https://blog.google/technology/developers/gemma-open-models/) using qLora and Supervise Finetuning

This is a comprahensive notebook and tutorial on how to fine tune the `gemma-2b-it` Model

## Prerequisites

Before delving into the fine-tuning process, ensure that you have the following prerequisites in place:

1. **GPU**: [gemma-2b](https://huggingface.co/google/gemma-2b) - can be finetuned on T4(free google colab) while [gemma-7b](https://huggingface.co/google/gemma-7b) requires an A100 GPU.
2. **Python Packages**: Ensure that you have the necessary Python packages installed. You can use the following commands to install them:

Let's begin by checking if your GPU is correctly detected:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/GEMMALLM

/content/drive/MyDrive/GEMMALLM


In [1]:
!nvidia-smi
!ls
!pwd

Fri Aug 23 02:50:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Step 2 - Model loading
We'll load the model using QLoRA quantization to reduce the usage of memory


In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatibl

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Now we specify the model ID and then we load it with our previously defined quantization configuration.Now we specify the model ID and then we load it with our previously defined quantization configuration.

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# model_id = "google/gemma-7b-it"

# model_id = "google/gemma-7b"
model_id = "google/gemma-2b-it"
# model_id = "google/gemma-2b"

In [5]:
# orginal_model = AutoModelForCausalLM.from_pretrained(model_id)
orginal_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    torch_dtype=torch.bfloat16
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
orginal_model.save_pretrained("./original_model")

In [10]:
def count_parameters(model):
    """Count the number of trainable parameters in the model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def calculate_model_size(model_dir):
    """Calculate the size of the saved model directory."""
    total_size = sum(os.path.getsize(os.path.join(model_dir, f)) for f in os.listdir(model_dir))
    return total_size / (1024 * 1024 * 1024)  # Size in GB


In [12]:
import os
#. Count trainable parameters and size of the original model
original_trainable_params = count_parameters(orginal_model)
original_model_size = calculate_model_size("/content/original_model")
print(f"Original Model Trainable Parameters: {original_trainable_params}")
print(f"Original Model Size: {original_model_size:.2f} GB")

Original Model Trainable Parameters: 2506172416
Original Model Size: 4.67 GB


In [6]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [14]:
model.save_pretrained("./quant_original_model")

In [15]:
#. Count trainable parameters and size of the original model
original_trainable_params = count_parameters(model)
original_model_size = calculate_model_size("/content/quant_original_model")
print(f"Original Model Trainable Parameters: {original_trainable_params}")
print(f"Original Model Size: {original_model_size:.2f} GB")
# 0524363776
# 2506172416

Original Model Trainable Parameters: 524363776
Original Model Size: 1.93 GB


In [7]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Translate German to English
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

  return (decoded)

In [8]:
result = get_completion(query='''Lieber Thomas!
Jetzt bist du weit weg! Ich bin sehr unglücklich! Wie geht es dir in Hamburg? Hast du nette Kollegen in der Bank? Ist der Chef nett? Hast du schon eine Wohnung? Ist die Wohnung teuer? Ich arbeite schon zwei Wochen im Goethe-Gymnasium in München.

Die Kollegen und Kolleginnen sind sehr freundlich. Die Schülerinnen und Schüler sind auch sehr nett. München ist schön! Das Wetter ist gut. Aber meine Katze "Mimi" ist krank! Das ist schrecklich. Heute Abend gehe ich ins Theater.

Herzliche Grüße''', model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Translate German to English
  Lieber Thomas!
Jetzt bist du weit weg! Ich bin sehr unglücklich! Wie geht es dir in Hamburg? Hast du nette Kollegen in der Bank? Ist der Chef nett? Hast du schon eine Wohnung? Ist die Wohnung teuer? Ich arbeite schon zwei Wochen im Goethe-Gymnasium in München.

Die Kollegen und Kolleginnen sind sehr freundlich. Die Schülerinnen und Schüler sind auch sehr nett. München ist schön! Das Wetter ist gut. Aber meine Katze "Mimi" ist krank! Das ist schrecklich. Heute Abend gehe ich ins Theater.

Herzliche Grüße
  
model


  Lieber Thomas!
Jetzt you bist weit weg! Ich bin sehr unglücklich! Wie geht es dir in Hamburg? Hast du nette Kollegen in der Bank? Ist der Chef nett? Hast du schon eine Wohnung? Ist die Wohnung teuer? Ich arbeite schon zwei Wochen im Goethe-Gymnasium in München.

Die Kollegen und Kolleginnen sind sehr freundlich. Die Schülerinnen und Schüler sind auch sehr nett. München ist schön! Das Wetter ist gut. Aber meine Katze "Mimi" ist krank! 

# Observation

Currently gemma-2b  model is not able to translate german to english

So we can fine tune this gemma

## Step 3 - Load dataset for finetuning

In [9]:
# !cd

In [10]:
from datasets import load_dataset

dataset = load_dataset("kaitchup/opus-German-to-English")
dataset

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/940304 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['text'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['text'],
        num_rows: 940304
    })
})

## Step 4 - Apply Lora  
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [12]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [13]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [14]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [15]:
modules = find_all_linear_names(model)
print(modules)

['down_proj', 'k_proj', 'gate_proj', 'o_proj', 'q_proj', 'v_proj', 'up_proj']


In [16]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [18]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.2f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.04%


## Step 5 - Run the training!

Setting the training arguments:
* for the reason of demo, we just ran it for few steps (100) just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
# import transformers

# tokenizer.pad_token = tokenizer.eos_token


# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=train_data,
#     eval_dataset=test_data,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         warmup_steps=0.03,
#         max_steps=100,
#         learning_rate=2e-4,
#         fp16=True,
#         logging_steps=1,
#         output_dir="outputs_mistral_b_finance_finetuned_test",
#         optim="paged_adamw_8bit",
#         save_strategy="epoch",
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )


### Fine-Tuning with qLora and Supervised Fine-Tuning

We're ready to fine-tune our model using qLora. For this tutorial, we'll use the `SFTTrainer` from the `trl` library for supervised fine-tuning. Ensure that you've installed the `trl` library as mentioned in the prerequisites.

In [20]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    dataset_text_field="text",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=50,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


## Lets start training

In [21]:
import time
start = time.process_time()
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
print("Time taken: ", time.process_time() - start)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
50,3.880000
100,3.044900


Time taken:  315.605691723


 Share adapters on the 🤗 Hub

In [24]:
new_model = "gemma-2b-mt-German-to-English"

In [25]:
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [ ]:
!git config --global credential.helper store

In [ ]:
!huggingface-cli repo create german2english  --type model

git version 2.34.1
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)

You are about to create neuronstarml/german2english
Proceed? [Y/n] Y
Y
Y








Your repo now lives at:
  https://huggingface.co/neuronstarml/german2english

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/neuronstarml/german2english



In [ ]:
!pip install huggingface_hub

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/neuronstarml/gemma-2b-mt-German-to-English/commit/81a84414708a9e4ad80b3f970170c1fe6b62021b', commit_message='Upload tokenizer', commit_description='', oid='81a84414708a9e4ad80b3f970170c1fe6b62021b', pr_url=None, pr_revision=None, pr_num=None)

## Test out Finetuned Model

In [28]:
# model_finetune_id = "neuronstarml/gemma-2b-mt-German-to-English"
model_finetune_id = "Venky2409/gemma-2b-mt-German-to-English"
model_finetune = AutoModelForCausalLM.from_pretrained(model_finetune_id, quantization_config=bnb_config, device_map={"":0})
tokenizer_finetune = AutoTokenizer.from_pretrained(model_finetune_id, add_eos_token=True)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

In [33]:
result = get_completion(query='''Lieber Thomas!
Jetzt bist du weit weg! Ich bin sehr unglücklich! Wie geht es dir in Hamburg? Hast du nette Kollegen in der Bank? Ist der Chef nett? Hast du schon eine Wohnung? Ist die Wohnung teuer? Ich arbeite schon zwei Wochen im Goethe-Gymnasium in München.

Die Kollegen und Kolleginnen sind sehr freundlich. Die Schülerinnen und Schüler sind auch sehr nett. München ist schön! Das Wetter ist gut. Aber meine Katze "Mimi" ist krank! Das ist schrecklich. Heute Abend gehe ich ins Theater.

Herzliche Grüße''', model=model_finetune, tokenizer=tokenizer_finetune)
print("Result: ", result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Result:  
  user
  Translate German to English
  Lieber Thomas!
Jetzt bist du weit weg! Ich bin sehr unglücklich! Wie geht es dir in Hamburg? Hast du nette Kollegen in der Bank? Ist der Chef nett? Hast du schon eine Wohnung? Ist die Wohnung teuer? Ich arbeite schon zwei Wochen im Goethe-Gymnasium in München.

Die Kollegen und Kolleginnen sind sehr freundlich. Die Schülerinnen und Schüler sind auch sehr nett. München ist schön! Das Wetter ist gut. Aber meine Katze "Mimi" ist krank! Das ist schrecklich. Heute Abend gehe ich ins Theater.

Herzliche Grüße
  
model


  .
* User-1955527382617854425174 14 June 2022

  <h1>Hello, Thomas!</h1>

  Der Titel wurde automatisch geändert, weil der Original-Beitrag in anderen Sprachen verfügbar ist. Klicken Sie auf links, um zwischen Übersetzungen zu wechseln.

  Dear Thomas!
  I hope you are away from heavy traffic. I am very unhappy! How are you in Hamburg? Do you have friendly colleagues in the bank? Is the boss nice? Have you a flat yet? Is it ex

In [45]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# !pip install langchain_community

# from langchain.schema import OutputParser
# from langchain.schema.output_parser import StringOutputParser
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
# parser = stringoutputparser() # This line seems to be a typo, as 'stringoutputparser' is not defined. Remove this line.

In [46]:
parser.invoke(result)

'\n  user\n  Translate German to English\n  Lieber Thomas!\nJetzt bist du weit weg! Ich bin sehr unglücklich! Wie geht es dir in Hamburg? Hast du nette Kollegen in der Bank? Ist der Chef nett? Hast du schon eine Wohnung? Ist die Wohnung teuer? Ich arbeite schon zwei Wochen im Goethe-Gymnasium in München.\n\nDie Kollegen und Kolleginnen sind sehr freundlich. Die Schülerinnen und Schüler sind auch sehr nett. München ist schön! Das Wetter ist gut. Aber meine Katze "Mimi" ist krank! Das ist schrecklich. Heute Abend gehe ich ins Theater.\n\nHerzliche Grüße\n  \nmodel\n\n\n  .\n* User-1955527382617854425174 14 June 2022\n\n  <h1>Hello, Thomas!</h1>\n\n  Der Titel wurde automatisch geändert, weil der Original-Beitrag in anderen Sprachen verfügbar ist. Klicken Sie auf links, um zwischen Übersetzungen zu wechseln.\n\n  Dear Thomas!\n  I hope you are away from heavy traffic. I am very unhappy! How are you in Hamburg? Do you have friendly colleagues in the bank? Is the boss nice? Have you a flat 


  user
  Translate German to English
  Lieber Thomas!
Jetzt bist du weit weg! Ich bin sehr unglücklich! Wie geht es dir in Hamburg? Hast du nette Kollegen in der Bank? Ist der Chef nett? Hast du schon eine Wohnung? Ist die Wohnung teuer? Ich arbeite schon zwei Wochen im Goethe-Gymnasium in München.

Die Kollegen und Kolleginnen sind sehr freundlich. Die Schülerinnen und Schüler sind auch sehr nett. München ist schön! Das Wetter ist gut. Aber meine Katze "Mimi" ist krank! Das ist schrecklich. Heute Abend gehe ich ins Theater.

Herzliche Grüße
  
model


  .
* User-1955527382617854425174 14 June 2022

  <h1>Hello, Thomas!</h1>

  Der Titel wurde automatisch geändert, weil der Original-Beitrag in anderen Sprachen verfügbar ist. Klicken Sie auf links, um zwischen Übersetzungen zu wechseln.

  Dear Thomas!
  I hope you are away from heavy traffic. I am very unhappy! How are you in Hamburg? Do you have friendly colleagues in the bank? Is the boss nice? Have you a flat yet? Is it expensive? I've been working two weeks at the Goethe Gymnasium in Munich.
  The colleagues are very friendly. The daughters are also nice. Munich is beautiful! The weather is good. But my cat "Mimi" is sick! This is terrible. Today I go to the theater.

  Good luck.
     kuah

  <h1>Glaub: Thomas !</h1>

  Der Titel wurde automatisch geändert, weil der Original-Beitrag in anderen Sprachen verfügbar ist. Klicken Sie auf links, um zwischen Übersetzungen zu wechseln.

  Hello Thomas!
  If you were away from heavy traffic. I'm very unhappy! How are you in Hamburg? Are you any friendly people in the bank? Is the boss nice? Have you already a flat? Is the flat expensive? I've been here two weeks at the Goethe-Gymnasium in Munich.

  The colleagues and colleagues are friendly. The pupils and pupils are also nice. Munich is beautiful! The weather is good. But my cat "Mimi" is sick! This is awful. I go to theater tonight.

  Best wishes
  Glaub: Thomas!

  <h1>Thomas!</h1>

  Der Titel wurde automatisch geändert, weil der Original-Beitrag in anderen Sprachen verfügbar ist. Klicken Sie auf links, um zwischen Übersetzungen zu wechseln.

  Hello Thomas!
  I hope you are away from heavy traffic. I am very unhappy! How are you in Hamburg? Are you any friendly people in the bank? Is the boss nice? Have you already flat? Is the apartment expensive? I've been here two weeks at the Goethe-Gymnasium in Munich.

  The colleagues and colleagues are friendly. The daughters are also nice. Munich is beautiful! The weather is good. But my cat "Mimi" is sick! This is terrible. Let's go to the cinema tonight.

  Good luck
  Thomas!

  <h1>Thomas !</h1>

  Der Titel wurde automatisch geändert, weil der Original-Beitrag in anderen Sprachen verfügbar ist. Klicken Sie auf links, um zwischen Übersetzungen zu wechseln.

  hello thomas!
  hope you are away from heavy traffic. I'm very unhappy! Do you have friendly colleagues in the bank? Is the boss nice? Do you already in flat? Is the flat expensive? I've been here two weeks at the Goethe-Gymnasium in Munich.

  the colleagues and colleagues are friendly. The students are also nice. Munich is beautiful! The weather is good. But my cat "Mimi" is sick! This is terrible. I go to the cinema tonight.

  Best wishes
  thomas!

* S6808252696002761799962 15 June 2022

  <h1>Hello, Thomas!</h1>

  Der Titel wurde automatisch geändert, weil der Original-Beitrag in anderen Sprachen verfügbar ist. Klicken Sie auf links, um zwischen Übersetzungen zu wechseln.

  Hi Thomas!
  I would have had to put a brake or just ignore the fact that I am going to your concert. My God! I hope your voice is still in order.

  I hope your children are happy, because they seem to be so happy. I hope you are happy with your friends and colleagues who are so nice to you:
  - The German guy next door: "Good day!" - The Danish neighbor across the street: "Good day !!" - The girl from the block next to my balcony: "Good day, Thomas! The German neighbor below your balcony: "Good day!" - Your sister next door: "Good day!" - The English man next door: "Good day!" - The Russian lady next door: "Hello, Thomas!"
  - I know you!
  - You are my best friend!
  - You are a super! You are the best!
  - You are a wonderful Thomas!

  And you like to hang out.

  I thought, you have to work at night.

  I wonder how you are on the weekend.

  The fact that you come to the lecture in the gym on Sunday!
  - It's always good to be